In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

# Denoising and Detoning (of Covariance Matrices)

This Jupyter Notebook contains a summary of denoising and detoning methods with example codes from Machine Learning for Asset Managers by Marcos Lopez de Prado

## Motivation

Empirical covariance matrices are computed on series of observations from a random vector, in order to estimate the linear comovement between the random variables that constitute the random vector.

Given the finite and nondeterministic nature of these observations, the estimate of the covariance matrix includes some amount of noise.

Empirical covariance matrices derived from estimated factors are also numerically ill-conditioned, because those factors are also estimated from flawed data. Unless we treat this noise, it will impact the calculations we perform with the covariance matrix, sometimes to the point of rendering the analysis useless.

## The Marcenko–Pastur Theorem

- Random Observations $x$ of size $T$ x $N$
- Underlying process generating observations with zero mean and variance $\sigma^2$
- $C = T^{-1} X' X$ has eigenvalues $\lambda$ that asymptotically follow the Marcenko–Pastur distribution

$$ f[\lambda] = \begin{cases} \frac{T}{N} \frac{\sqrt{(\lambda_+ - \lambda)(\lambda - \lambda_-)}}{2\pi\lambda\sigma^2} & \text{if } \lambda \in [\lambda_- , \lambda_+] \\ 0 & \text{if } \lambda \notin [\lambda_- , \lambda_+] \end{cases} $$

With expected maximum eigenvalue $\lambda_+ = (1 + \sqrt{\frac{N}{T}})^2 \sigma^2$ and expected minimum eigenvalue $\lambda_- = (1 - \sqrt{\frac{N}{T}})^2 \sigma^2$

Eigenvalues $\lambda \in [\lambda_- , \lambda_+]$ are consistent with random behavior, and eigenvalues $\lambda \notin [\lambda_- , \lambda_+]$ are consistent with non-random behavior. Specifically we associate eigenvalues $\lambda \in [0, \lambda_+]$ with noise.

In [ ]:
import numpy as np,pandas as pd

def mpPDF(var,q,pts):
    # Marcenko-Pastur pdf
    # q=T/N
    eMin,eMax = var * (1 - (1. / q)**.5)**2, var * (1 + (1. / q)**.5)**2
    eVal = np.linspace(eMin, eMax, pts)
    pdf = q / (2 * np.pi * var * eVal) * ((eMax - eVal) * (eVal - eMin))**.5
    pdf = pd.Series(pdf, index=eVal)
    return pdf

Testing the Marcenko–Pastur Theorem

In [ ]:
from sklearn.neighbors import KernelDensity

def getPCA(matrix):
    # Get eVal, eVec from a Hermitian matrix
    eVal, eVec = np.linalg.eigh(matrix)
    indices = eVal.argsort()[::-1] # arguments for sorting eVal desc
    eVal, eVec = eVal[indices], eVec[:,indices]
    eVal = np.diagflat(eVal)
    return eVal, eVec

def fitKDE(obs, bWidth=.25, kernel='gaussian', x=None):
    # Fit kernel to a series of obs, and derive the prob of obs
    # x is the array of values on which the fit KDE will be evaluated
    if len(obs.shape) == 1:
        obs=obs.reshape(-1,1)
    
    kde = KernelDensity(kernel=kernel,bandwidth=bWidth).fit(obs)
    
    if x is None:
        x = np.unique(obs).reshape(-1,1)
    
    if len(x.shape) == 1:
        x = x.reshape(-1,1)
    logProb = kde.score_samples(x) # log(density)
    pdf = pd.Series(np.exp(logProb), index=x.flatten())
    return pdf

# Test the above functions
x = np.random.normal(size=(10000,1000))
eVal0, eVec0 = getPCA(np.corrcoef(x, rowvar=0))
pdf0 = mpPDF(1.,q=x.shape[0] / float(x.shape[1]), pts=1000)
pdf1 = fitKDE(np.diag(eVal0), bWidth=.01) # empirical pdf

In [ ]:
# Plotting
fig = go.Figure()
fig.add_trace(go.Scatter(x=pdf1.index, y=pdf1.values, name='Empirical'))
fig.add_trace(go.Scatter(x=pdf0.index, y=pdf0.values, name='Marcenko-Pastur'))
fig.update_layout(title='Marcenko-Pastur vs Empirical PDF',
                  xaxis_title='Eigenvalue λ',
                  yaxis_title='Probability[λ]',
                  width=800,
                  height=800,
                  xaxis_range=[0, 2]
                  )
fig.show()

## Fitting the Marcenko–Pastur Distribution

The goal is to discriminate between eigenvalues attributed to signal and noise

In [ ]:
# Functions to add signal to noise

def getRndCov(nCols,nFacts):
    w = np.random.normal(size=(nCols, nFacts))
    cov = np.dot(w,w.T) # random cov matrix, however not full rank
    cov += np.diag(np.random.uniform(size=nCols)) # full rank cov
    return cov

def cov2corr(cov):
    # Derive the correlation matrix from a covariance matrix
    std = np.sqrt(np.diag(cov))
    corr = cov / np.outer(std,std)
    corr[corr<-1], corr[corr>1]=-1, 1 # numerical error
    return corr


In [ ]:
alpha, nCols, nFact, q = .995, 1000, 100, 10
cov = np.cov(np.random.normal(size=(nCols*q,nCols)), rowvar=0)
cov = alpha * cov + (1 - alpha) * getRndCov(nCols, nFact) # noise+signal
corr0 = cov2corr(cov)
eVal0,eVec0 = getPCA(corr0)

Fitting the PDF by minimizing squared difference between analytical marcenko-pastur distribution and kernel density estimate of eigenvalues

In [ ]:
def errPDFs(var,eVal,q,bWidth,pts=1000):
    # Fit error

    # scipy minimize puts all vars in a vector, so define a function to unpack 
    if type(var) == np.ndarray:
        var = var[0]

    pdf0 = mpPDF(var, q, pts) # theoretical pdf
    pdf1 = fitKDE(eVal, bWidth, x=pdf0.index.values) # empirical pdf
    sse = np.sum((pdf1 - pdf0)**2)
    return sse

In [ ]:
from scipy.optimize import minimize

def findMaxEval(eVal, q, bWidth):
    
    # Find max random eVal by fitting Marcenko’s dist
    out = minimize(lambda *x: errPDFs(*x), .5, args=(eVal, q, bWidth), bounds=((1E-5, 1-1E-5),))
    
    if out['success']:
        var = out['x'][0]
    else:
        var = 1
    eMax = var * (1 + (1. / q)**.5)**2
    return eMax, var

Output:
- eigenvalue cutoff $\lambda_+$
- noise variance $\sigma^2$

In [ ]:
# fitting the marcenko-pastur distribution
eMax0, var0 = findMaxEval(np.diag(eVal0), q, bWidth=.01)
nFacts0 = eVal0.shape[0] - np.diag(eVal0)[::-1].searchsorted(eMax0)

# compute pdfs for plotting
eValArr = np.diag(eVal0)

pdf0 = mpPDF(var0, q=nCols/nFacts0, pts=1000)
pdf1 = fitKDE(eValArr, bWidth=.01)

# compute histogram of eValArr
hist, bins = np.histogram(eValArr, bins=1000, density=True)


In [ ]:

print("Variance that is explained by the random eigenvectors: ", var0)
print("Cutoff level for eigenvalues: ", eMax0)
print("Percent of variance attributed to signal: ", 1-var0)
print("Signal to noise ratio: ", (1-var0)/var0)

# Plotting
fig = go.Figure()
fig.add_trace(go.Scatter(x=bins, y=hist, name='Empirical (Histogram)'))
fig.add_trace(go.Scatter(x=pdf0.index, y=pdf0.values, name='Marcenko-Pastur'))
fig.add_trace(go.Scatter(x=pdf1.index, y=pdf1.values, name='Empirical'))
fig.update_layout(title='Empirical PDF of Eigenvalues',
                  xaxis_title='Eigenvalue λ',
                  yaxis_title='Probability[λ]',
                  width=1000,
                  height=800,
                  xaxis_range=[0, 7]
                  )
fig.show()


## Denoising

Methods:
- Constant Residual Eigenvalue
- Shrinkage

Problem:
- Noise and Signal are both shrinked by the same amount so it can make a weak signal disappear

### Constant Residual Eigenvalue

In [ ]:
def denoisedCorr(eVal,eVec,nFacts):
    # Remove noise from corr by fixing random eigenvalues
    eVal_=np.diag(eVal).copy()
    eVal_[nFacts:]=eVal_[nFacts:].sum() / float(eVal_.shape[0]-nFacts)
    eVal_=np.diag(eVal_)
    corr1=np.dot(eVec,eVal_).dot(eVec.T)
    corr1=cov2corr(corr1)
    return corr1

In [ ]:
corr1 = denoisedCorr(eVal0, eVec0, nFacts0)
eVal1, eVec1 = getPCA(corr1)

In [ ]:
# plot eigenvalues
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(eVal0)), y=np.log(np.diag(eVal0)), name='Original'))
fig.add_trace(go.Scatter(x=np.arange(len(eVal1)), y=np.log(np.diag(eVal1)), name='Denoised'))
fig.update_layout(title='Eigenvalues of Correlation Matrix',
                  xaxis_title='Eigenvalue Index',
                  yaxis_title='Eigenvalue (log-scaled)',
                  width=1000,
                  height=800,
                  )
fig.show()

In [ ]:
from numpy import linalg as LA
print(LA.cond(corr0))
print(LA.cond(corr1))

### Denoised by Targeted Shrinkage

In [ ]:
def denoisedCorr2(eVal,eVec,nFacts,alpha=0):
    # Remove noise from corr through targeted shrinkage
    eValL,eVecL=eVal[:nFacts,:nFacts],eVec[:,:nFacts]
    eValR,eVecR=eVal[nFacts:,nFacts:],eVec[:,nFacts:]
    corr0=np.dot(eVecL,eValL).dot(eVecL.T)
    corr1=np.dot(eVecR,eValR).dot(eVecR.T)
    corr2=corr0+alpha*corr1+(1-alpha)*np.diag(np.diag(corr1))
    return corr2


In [ ]:
corr1 = denoisedCorr2(eVal0,eVec0,nFacts0,alpha=.5)
eVal1, eVec1 = getPCA(corr1)

In [ ]:
# plot eigenvalues
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(eVal0)), y=np.log(np.diag(eVal0)), name='Original'))
fig.add_trace(go.Scatter(x=np.arange(len(eVal1)), y=np.log(np.diag(eVal1)), name='Denoised'))
fig.update_layout(title='Eigenvalues of Correlation Matrix',
                  xaxis_title='Eigenvalue Index',
                  yaxis_title='Eigenvalue (log-scaled)',
                  width=1000,
                  height=800,
                  )
fig.show()

## Detoning

Goal is to remove a market component which affects all other eigenvalues. Important for clustering where the algorithm struggles to find dissimilarities between clusters.

Detoning is the principal components analysis analogue to computing beta-adjusted (or market-adjusted) returns in regression analysis.

In [ ]:
# not interesting for me at the moment maybe implement later